In [53]:
import pandas as pd

In [54]:
#dtype={"user_id": int, "username": object}

In [55]:
inspections = pd.read_csv('data/potential_inspection_summary_data.csv', sep=';')
violations = pd.read_csv('data/potential_violation_details_data.csv', sep=';')
locations = pd.read_csv('data/inspections_geocoded.csv', sep=';')

In [62]:
# Drop entries that are exact duplicates, ignoring the 'inspection_id' column
inspections = inspections.drop_duplicates(inspections.columns.difference(['inspection_id']))

In [63]:
# Only include food establishments
# The inspections table includes non-food places like pools and barber shops
food_establishment_types = ['Bakery', 'Caterer', 'Commission Merchant',
                            'Delicatessen', 'Food Products', 'Grocery Store',
                            'Hotel', 'Ice Cream Manufacturer', 'Marine-Food (Wholesale)',
                            'Marine-Food Prod (Retail)', 'Mobile Vending',
                            'Restaurant Total', 'School Cafeteria']
food_inspections = inspections[inspections['establishment_type'].isin(food_establishment_types)]

# Make a table of food inspetions that are known to be valid entries
known_inspections = food_inspections[food_inspections['known_valid'] == True]
#len(known_inspections)

In [64]:
# food_inspections['establishment_type'].value_counts()

In [82]:
# Look at cases where a place (defined by license_number) had multple visits in a day
# Two categories: those with same start time should be flagged as duplicates,
# while those with different ones are the interesting group of places that got two visits in a day
license_day_groups = known_inspections.groupby(['license_number', 'inspection_date'])
for name, group in license_day_groups:
    if len(group) > 1:
        print name, len(group)
        for i, j in group.iterrows():
            print j['inspection_id'], '-', j['inspection_time_in'], j['inspection_time_out']

('00000XXXX-12000075', '26/04/2012') 2
122812 - 12:02 PM 12:55 PM
142429 - 2:33 PM 2:38 PM
('00000XXXX-65001280', '23/02/2011') 2
116086 - 10:30 AM nan
116088 - 10:30 AM 11:20 AM
('00000xxxx-53004850', '18/03/2011') 2
21197 - 1:05 PM 1:50 PM
94114 - 1:05 PM 1:50 PM
('00013xxxx-00000205', '28/10/2014') 2
405271 - 11:00 AM 11:35 AM
405272 - 11:00 AM 11:35 AM
('00013xxxx-00000326', '25/03/2015') 2
610938 - 10:40 AM 11:20 AM
610939 - 10:40 AM 11:20 AM
('00294xxxx-130000223', '10/11/2016') 2
833230 - 2:00 PM 2:30 PM
833231 - 2:00 PM 2:30 PM
('02066xxxx-00000093', '21/09/2016') 2
832024 - 11:05 AM 11:40 AM
832028 - 12:45 PM 12:55 PM
('03011xxxx-13000009', '03/05/2016') 2
824281 - 3:00 PM 4:00 PM
824282 - 3:00 PM 4:00 PM
('05184xxxx-39802487', '28/06/2011') 2
21162 - 12:10 PM 12:30 PM
21169 - 11:30 AM 12:00 PM
('09113xxxx-64002072', '17/01/2014') 2
201160 - 11:00 AM 11:50 AM
201169 - 11:00 AM 12:00 PM
('09303xxxx-12000009', '01/03/2013') 2
177341 - 10:35 AM nan
177342 - 10:35 AM 11:40 PM
('09

In [93]:
count_total = 0
count_multiple_times = 0
count_hot_water = 0
for name, group in license_day_groups:
    if len(group) > 1:
        count_total += 1
        if len(group['inspection_time_in'].unique()) > 1:
            count_multiple_times += 1
            no_hot_water = False
            for id_ in group['inspection_id']:
                matching_violations = violations[violations['inspection_id'] == id_]
                #print list(matching_violations['violation_number'])
                if 48 in list(matching_violations['violation_number']):
                    no_hot_water = True
            if no_hot_water:
                count_hot_water += 1
            #print name
print count_total
print count_multiple_times
print count_hot_water

285
138
30
